## Meta Analysis

A notebook to analyze the results of Population, the genetic algorithm. See examples/run_meta.

In [ ]:
import os
import numpy as np
from spikey.logging import MetaReader
import matplotlib.pyplot as plt

In [ ]:
# Folder containing population log files.
RESULTS_FOLDER = os.path.join('..', 'log', 'metarl')

In [ ]:
reader = MetaReader(folder=RESULTS_FOLDER)
reader.summary

In [ ]:
def iter_epoch(dataframe, row_per_epoch):
    # Ceil division - 1
    for i in range((len(dataframe) - 1) // row_per_epoch + 1 - 1):
        yield dataframe[i * row_per_epoch:(i + 1) * row_per_epoch]

In [ ]:
N_AGENTS = reader.summary['info']['population_config']['n_agents']
N_REPEATS = reader.summary['info']['metagame_info']['n_reruns']
df = reader.df

ep_fitness = []
for i, episode in enumerate(iter_epoch(df, N_AGENTS)):
    ep_fitness.append(np.copy(episode['fitness']))

In [ ]:
# Agent fitnesses per epoch
for i in range(len(ep_fitness) // 10 - 1):
    plt.violinplot(ep_fitness[i * 10:(i + 1) * 10])
    plt.title(f"Epoch {i * 10}-{(i + 1) * 10}")
    plt.show()

In [ ]:
# Agent fitness percentiles per epoch
for percentile in [50, 75, 95, 100]:
    X, Y_a = [], []
    for i in range(len(ep_fitness)):
        X.append(i)
        Y_a.append(np.percentile(ep_fitness[i], percentile))

    plt.title(f"{percentile}th percentile of fitnesses")
    plt.scatter(X, Y_a, label="Fitness")
    plt.legend()
    plt.show()

In [ ]:
# Genotype(s) with highest fitness
df = reader.df
params = df[df['fitness'] == np.max(df['fitness'])]
filenames = df[df['fitness'] == np.max(df['fitness'])]['filename']

print(np.max(df['fitness']))
for i, filename in enumerate(filenames):
    print(params.iloc[i])